<a href="https://colab.research.google.com/github/fatemafaria142/From-Text-to-Code-Generation-Pipeline-Incorporating-Various-LLMs/blob/main/Code_Generation_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("sahil2801/CodeAlpaca-20k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains three different columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 20022
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")


Data Point 1:
Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
Input: 
Output: arr = [2, 4, 6, 8, 10]

-----------------------------

Data Point 2:
Instruction: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
Input: 
Output: Height of triangle = opposite side length * sin (angle) / side length

-----------------------------

Data Point 3:
Instruction: Write a replace method for a string class which replaces the given string with a given set of characters.
Input: string = "Hello World!"
replace_with = "Greetings!"
Output: def replace(self, replace_with):
    new_string = ""
    for char in self:
        if char == " ":
            new_string += replace_with
        else:
            new_string += char
    return new_string

-----------------------------

Data Point 4:
Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
Input: 
Output: arr = [3, 

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 300
    })
})

In [36]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt= " Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: "
    prompt += sample['instruction']
    prompt += sample['input']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {sample['output']}"
    return single_turn_prompt

In [37]:
create_prompt(instruct_tune_dataset["train"][0])

"GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Create an array of length 5 which contains all even numbers between 1 and 10.<|end_of_turn|>GPT4 Correct Assistant: arr = [2, 4, 6, 8, 10]"

In [38]:
create_prompt(instruct_tune_dataset["train"][1])

"GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.<|end_of_turn|>GPT4 Correct Assistant: Height of triangle = opposite side length * sin (angle) / side length"

In [39]:
create_prompt(instruct_tune_dataset["train"][2])

'GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you\'re ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Write a replace method for a string class which replaces the given string with a given set of characters.string = "Hello World!"\nreplace_with = "Greetings!"<|end_of_turn|>GPT4 Correct Assistant: def replace(self, replace_with):\n    new_string = ""\n    for char in self:\n        if char == " ":\n            new_string += replace_with\n        else:\n            new_string += char\n    return new_string'

In [40]:
create_prompt(instruct_tune_dataset["train"][3])

"GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Create an array of length 15 containing numbers divisible by 3 up to 45.<|end_of_turn|>GPT4 Correct Assistant: arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45]"

In [41]:
create_prompt(instruct_tune_dataset["train"][4])

'GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you\'re ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Write a function to find the number of distinct states in a given matrix.matrix = [[1, 0, 0],\n          [1, 0, 1],\n          [1, 1, 1]]<|end_of_turn|>GPT4 Correct Assistant: def find_num_distinct_states(matrix):\n    states = set()\n    for row in matrix:\n        state = "".join([str(x) for x in row])\n        states.add(state)\n    return len(states)'

In [42]:
create_prompt(instruct_tune_dataset["train"][5])

"GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Create a nested loop to print every combination of numbers between 0-9<|end_of_turn|>GPT4 Correct Assistant: for i in range(10):\n    for j in range(10):\n        print(i, j)"

In [43]:
create_prompt(instruct_tune_dataset["train"][6])

"GPT4 Correct User:  Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Write a function to find the maximum difference between two numbers in a given array.arr = [5, 3, 17, 11, 9]<|end_of_turn|>GPT4 Correct Assistant: def max_difference(arr):\n    max_diff = 0\n    for i in range(len(arr)-1):\n        diff = arr[i+1] - arr[i]\n        if diff > max_diff:\n            max_diff = diff\n    return max_diff"

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [45]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [46]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [47]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [53]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [56]:
prompt = """GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: """
prompt += '''Create an array of length 15 containing numbers divisible by 3 up to 45.<|end_of_turn|> '''
prompt += "GPT4 Correct Assistant: "

In [57]:
print(prompt)

GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Create an array of length 15 containing numbers divisible by 3 up to 45.<|end_of_turn|> GPT4 Correct Assistant: 


In [58]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s> GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Create an array of length 15 containing numbers divisible by 3 up to 45.<|end_of_turn|>  GPT4 Correct Assistant: \n\nTo solve this coding problem, we need to create an array with a length of 15 that contains numbers divisible by 3 up to 45. \n\nHere's a Python solution:\n\n```python\n# Initialize an empty array\narray = []\n\n# Iterate through the range of numbers from 1 to 45 (inclusive) in steps of 3\n# and append each number to the array if it is divisible by 3\nfor i in range(1, 46, 3):\n    if i % 3 == 0:\n        array.append(i)\n        \n# Print the resulting array\nprint(array)\n```\n\nWhen you run this code, you'l

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [59]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [60]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [61]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=200,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [62]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [63]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.023800
20,0.570600
30,0.443700
40,0.457400
50,0.442200
60,0.434900
70,0.413000
80,0.406800
90,0.404400
100,0.434200


TrainOutput(global_step=200, training_loss=0.452457457780838, metrics={'train_runtime': 1740.8851, 'train_samples_per_second': 0.46, 'train_steps_per_second': 0.115, 'total_flos': 1.7483553570816e+16, 'train_loss': 0.452457457780838, 'epoch': 0.65})

### **Save the model**

In [64]:
trainer.save_model("Starling_instruct_generation")

In [65]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [66]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [67]:
prompt = "GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: "
prompt += "Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length. <|end_of_turn|>  GPT4 Correct Assistant: Given angle, side lengths and opposite side length, the equation to calculate the height of a triangle is:

height = (side² × angle)/(2 × opposite side length)

where height is the height of the triangle, side is any side length of the triangle, angle is the angle at which the opposite side length is to the side, and opposite side length is the length of the opposite side to the angle.<|end_of_turn|>


### **Example No:2**

In [68]:
prompt = "GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: "
prompt += '''Write a Python function to replace all the occurrences of a given substring with another. string = "Hello World"
substring = "World"
replacement = "Universe" <|end_of_turn|> '''
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Write a Python function to replace all the occurrences of a given substring with another. string = "Hello World"
substring = "World"
replacement = "Universe" <|end_of_turn|>  GPT4 Correct Assistant: def replace_substring(string, substring, replacement):
    return string.replace(substring, replacement)

result = replace_substring(string, substring, replacement)
print(result)  # Output: "Hello Universe"<|end_of_turn|>


### **Example No:3**

In [69]:
prompt = "GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: "
prompt += "Find the smallest element in a given array.[6, 8, 1, 4, 5] <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Think of yourself as a Coding Expert. Your expertise lies in solving coding problems with precision and clarity. Imagine you're ready to provide accurate solutions to any coding-related inquiry. Your goal is to solve the given coding problem with utmost accuracy. Please proceed with the solution once the question is provided. Coding Question: Find the smallest element in a given array.[6, 8, 1, 4, 5] <|end_of_turn|>  GPT4 Correct Assistant: smallest_element = min(array)
print(smallest_element)<|end_of_turn|>
